## 准备工作

In [4]:
import numpy as np

from utils import create_base_network, ensure_weak_connectivity, adjacency_to_incidence, create_commodities

In [5]:
N = 5; k = 2
seed = 1
W = create_base_network(N, k, seed)
W = ensure_weak_connectivity(W, seed)
W

array([[0.        , 2.83454341, 1.72628826, 0.        , 0.        ],
       [1.06832089, 0.        , 3.07184082, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 1.42058719, 1.00499716],
       [0.        , 0.        , 1.26497805, 0.        , 0.6807877 ],
       [0.        , 0.        , 0.79877644, 0.91470983, 0.        ]])

In [6]:
A, c = adjacency_to_incidence(W)
A, c

(array([[-1., -1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  0., -1., -1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  1., -1., -1.,  1.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  0.,  1.,  0., -1., -1.,  0.,  1.],
        [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  1., -1., -1.]]),
 array([2.83454341, 1.72628826, 1.06832089, 3.07184082, 1.42058719,
        1.00499716, 1.26497805, 0.6807877 , 0.79877644, 0.91470983]))

In [7]:
K = 3
commodities = create_commodities(W, K, 10.0, seed)
commodities

[(2, 3, 9.537845024235194),
 (1, 0, 4.809938040753181),
 (3, 2, 8.449323344383977)]

In [8]:
# 测试 spmv

# 先将邻接矩阵改为无权图
M = ((W > 0) & (W < np.inf)).astype(int)
print(M)

vec = np.array([1, 0, 0, 0, 1])

np.dot(M.T, vec), np.dot(W.T, vec)

[[0 1 1 0 0]
 [1 0 1 0 0]
 [0 0 0 1 1]
 [0 0 1 0 1]
 [0 0 1 1 0]]


(array([0, 1, 2, 1, 0]),
 array([0.        , 2.83454341, 2.5250647 , 0.91470983, 0.        ]))

## spmm最短路计算 不带路径

In [6]:
import numpy as np

def min_plus_matmul(W, D):
    """
    执行基于 (min, +) 半环的矩阵乘法 a ⊕.⊗ b
    即: Result(i, j) = min_k { W(i, k) + D(k, j) }
    
    参数:
    W (np.array): 权重矩阵 (N x N)
    D (np.array): 当前的距离矩阵 (N x N)
    
    返回:
    np.array: 新的距离矩阵 (N x N)
    """
    N = W.shape[0]
    # 初始化结果矩阵为无穷大
    D_new = np.full((N, N), np.inf)

    # W(i, k) + D(k, j)
    # 通过广播，我们可以一次性计算所有k的 W(i, k) + D(k, j)
    # W[:, np.newaxis, :] -> shape (N, 1, N)
    # D[np.newaxis, :, :] -> shape (1, N, N)
    # W[:, np.newaxis, :] + D[np.newaxis, :, :] -> shape (N, N, N)
    # 结果的 [i, j, k] 元素是 W[i, k] + D[k, j]
    # 然后我们在 k 轴上取最小值
    
    # 为了避免 NumPy 的警告，我们这里使用循环实现，逻辑更清晰
    # 虽然广播更快，但循环更容易理解
    for i in range(N):
        for j in range(N):
            # 计算 Result(i, j) = min(W[i,:] + D[:,j])
            # W[i, :] 是第 i 行
            # D[:, j] 是第 j 列
            # 它们相加后得到一个向量，取其最小值
            D_new[i, j] = np.min(W[i, :] + D[:, j])
            
    return D_new

def apsp_with_spmm(W):
    """
    使用基于 (min, +) 半环的矩阵乘法计算所有节点对的最短路径 (APSP)
    
    参数:
    W (np.array): 输入的邻接权重矩阵。0表示边不存在。
    
    返回:
    np.array: 包含所有节点对最短路径的最终距离矩阵。
    """
    N = W.shape[0]
    
    # 1. 准备权重矩阵 W_prime
    # 将权重矩阵中的 0 (表示无直连边) 替换为无穷大，对角线替换为 0
    W_prime = np.where(W == 0, np.inf, W)
    np.fill_diagonal(W_prime, 0)
    
    print("--- 初始化的权重矩阵 W' (∞表示无连接) ---")
    print(W_prime)
    
    # 2. 初始化距离矩阵 D
    # D_0 就是 W_prime，因为它代表了长度为1的路径
    D = W_prime.copy()
    
    # 3. 迭代执行 SpMM
    # 算法最多在 N-1 轮后收敛（如果没有负权环）
    for i in range(1, N):
        print(f"\n--- 第 {i} 轮迭代 ---")
        D_old = D.copy()
        
        # 核心操作: D_new = W' ⊕.⊗ D_old
        D_candidates = min_plus_matmul(W_prime, D_old)
        
        # 将新发现的更短路径合并到结果中
        # D = min(D_old, D_candidates)
        D = np.minimum(D_old, D_candidates)
        
        print("当前最短路径矩阵 D:")
        print(D)
        
        # 4. 检查收敛
        if np.array_equal(D, D_old):
            print(f"\n矩阵在第 {i} 轮迭代后收敛，算法结束。")
            break
            
    return D

In [7]:
final_distances = apsp_with_spmm(W.T)
print(f"\n从节点 A 到节点 D 的最短路径是: {final_distances[4, 0]:.4f}")

--- 初始化的权重矩阵 W' (∞表示无连接) ---
[[0.         1.06832089        inf        inf        inf]
 [2.83454341 0.                inf        inf        inf]
 [1.72628826 3.07184082 0.         1.26497805 0.79877644]
 [       inf        inf 1.42058719 0.         0.91470983]
 [       inf        inf 1.00499716 0.6807877  0.        ]]

--- 第 1 轮迭代 ---
当前最短路径矩阵 D:
[[0.         1.06832089        inf        inf        inf]
 [2.83454341 0.                inf        inf        inf]
 [1.72628826 2.79460915 0.         1.26497805 0.79877644]
 [3.14687545 4.49242801 1.42058719 0.         0.91470983]
 [2.73128542 4.07683798 1.00499716 0.6807877  0.        ]]

--- 第 2 轮迭代 ---
当前最短路径矩阵 D:
[[0.         1.06832089        inf        inf        inf]
 [2.83454341 0.                inf        inf        inf]
 [1.72628826 2.79460915 0.         1.26497805 0.79877644]
 [3.14687545 4.21519634 1.42058719 0.         0.91470983]
 [2.73128542 3.79960631 1.00499716 0.6807877  0.        ]]

--- 第 3 轮迭代 ---
当前最短路径矩阵 D:
[[0.       

## spmm最短路计算 带路径

In [31]:
import numpy as np
def min_plus_matmul_with_predecessor(W, D, P_old):
    """
    执行基于 (min, +) 半环的矩阵乘法，同时维护前驱节点矩阵
    
    参数:
    W (np.array): 权重矩阵 (N x N)
    D (np.array): 当前的距离矩阵 (N x N)
    P_old (np.array): 当前的前驱节点矩阵 (N x N)
    
    返回:
    tuple: (D_new, P_new) 新的距离矩阵和前驱矩阵
    """
    N = W.shape[0]
    D_new = np.full((N, N), np.inf)
    P_new = P_old.copy()
    
    for i in range(N):
        for j in range(N):
            # 计算所有可能的路径长度: W[i, k] + D[k, j]
            # 这表示从源点 j 经过中间节点 k 到达节点 i 的路径长度
            path_costs = W[i, :] + D[:, j]
            
            # 找到最小值（最短路径长度）
            min_cost = np.min(path_costs)
            D_new[i, j] = min_cost
            
            # 找到使得路径最短的中间节点 k
            # 这个 k 就是从源点 j 到节点 i 的最短路径上，i 的直接前驱
            best_k = np.argmin(path_costs)
            
            # 更新前驱节点（只在路径可达时更新）
            if min_cost < np.inf:
                P_new[i, j] = best_k
                
    return D_new, P_new
def apsp_with_spmm_and_path(W):
    """
    使用基于 (min, +) 半环的矩阵乘法计算所有节点对的最短路径 (APSP)
    同时记录路径信息
    
    参数:
    W (np.array): 输入的邻接权重矩阵。0表示边不存在。
    
    返回:
    tuple: (D, P) 最终距离矩阵和前驱节点矩阵
    """
    N = W.shape[0]
    
    # 1. 准备权重矩阵 W_prime
    W_prime = np.where(W == 0, np.inf, W)
    np.fill_diagonal(W_prime, 0)
    
    print("--- 初始化的权重矩阵 W' ---")
    print(W_prime)
    print()
    
    # 2. 初始化距离矩阵 D
    D = W_prime.copy()
    
    # 3. 初始化前驱节点矩阵 P
    # P[i, j] 表示从源点 j 到节点 i 的路径上，i 的直接前驱节点
    P = np.full((N, N), -1, dtype=int)  # -1 表示不可达
    
    # 初始化规则：
    # - 如果有直接的边从 j 到 i (W_prime[i, j] < inf)，则前驱就是 j 自己
    # - 对角线上，节点到自身的前驱是自己
    for i in range(N):
        for j in range(N):
            if W_prime[i, j] < np.inf:
                P[i, j] = j  # 从 j 可以直接到达 i
    
    print("--- 初始化的前驱节点矩阵 P ---")
    print(P)
    print()
    
    # 4. 迭代执行 SpMM
    for iteration in range(1, N):
        print(f"{'='*60}")
        print(f"第 {iteration} 轮迭代")
        print(f"{'='*60}")
        
        D_old = D.copy()
        P_old = P.copy()
        
        # 核心操作: (D_new, P_new) = W' ⊕.⊗ (D_old, P_old)
        D_candidates, P_candidates = min_plus_matmul_with_predecessor(W_prime, D_old, P_old)
        
        # 将新发现的更短路径合并到结果中
        # 只有当新路径更短时，才更新距离和前驱
        for i in range(N):
            for j in range(N):
                if D_candidates[i, j] < D[i, j]:
                    D[i, j] = D_candidates[i, j]
                    P[i, j] = P_candidates[i, j]
        
        print("当前最短路径距离矩阵 D:")
        print(D)
        print("\n当前前驱节点矩阵 P:")
        print(P)
        print()
        
        # 5. 检查收敛
        if np.array_equal(D, D_old):
            print(f"✓ 矩阵在第 {iteration} 轮迭代后收敛，算法结束。\n")
            break
            
    return D, P
def reconstruct_path(P, src, dst):
    """
    根据前驱节点矩阵重建从 src 到 dst 的最短路径
    
    参数:
    P (np.array): 前驱节点矩阵
    src (int): 源节点
    dst (int): 目标节点
    
    返回:
    list: 从 src 到 dst 的路径节点列表，如果不可达则返回 None
    """
    if P[dst, src] == -1:
        return None  # 不可达
    
    # 从目标节点反向追溯到源节点
    path = []
    current = dst
    
    # 防止无限循环（如果有负环或者矩阵有问题）
    visited = set()
    max_iterations = P.shape[0] + 1
    
    while current != src and len(visited) < max_iterations:
        if current in visited:
            print(f"警告：检测到循环，路径重建失败")
            return None
        visited.add(current)
        
        path.append(current)
        # 查找当前节点的前驱
        current = P[current, src]
        
        if current == -1:
            return None  # 路径中断，不应该发生
    
    path.append(src)
    path.reverse()  # 反转得到从 src 到 dst 的正向路径
    
    return path
def print_path_with_weights(W_original, path):
    """
    打印路径及其边权重
    
    参数:
    W_original (np.array): 原始权重矩阵
    path (list): 路径节点列表
    """
    if path is None or len(path) < 2:
        return
    
    segments = []
    for i in range(len(path) - 1):
        from_node = path[i]
        to_node = path[i + 1]
        weight = W_original[to_node, from_node]
        segments.append(f"{from_node} --({weight:.3f})--> {to_node}")
    
    return " → ".join([str(path[0])] + [str(p) for p in path[1:]])


In [32]:
final_D, final_P = apsp_with_spmm_and_path(W.T)

--- 初始化的权重矩阵 W' ---
[[0.         1.06832089        inf        inf        inf]
 [2.83454341 0.                inf        inf        inf]
 [1.72628826 3.07184082 0.         1.26497805 0.79877644]
 [       inf        inf 1.42058719 0.         0.91470983]
 [       inf        inf 1.00499716 0.6807877  0.        ]]

--- 初始化的前驱节点矩阵 P ---
[[ 0  1 -1 -1 -1]
 [ 0  1 -1 -1 -1]
 [ 0  1  2  3  4]
 [-1 -1  2  3  4]
 [-1 -1  2  3  4]]

第 1 轮迭代
当前最短路径距离矩阵 D:
[[0.         1.06832089        inf        inf        inf]
 [2.83454341 0.                inf        inf        inf]
 [1.72628826 2.79460915 0.         1.26497805 0.79877644]
 [3.14687545 4.49242801 1.42058719 0.         0.91470983]
 [2.73128542 4.07683798 1.00499716 0.6807877  0.        ]]

当前前驱节点矩阵 P:
[[ 0  1 -1 -1 -1]
 [ 0  1 -1 -1 -1]
 [ 0  0  2  3  4]
 [ 2  2  2  3  4]
 [ 2  2  2  3  4]]

第 2 轮迭代
当前最短路径距离矩阵 D:
[[0.         1.06832089        inf        inf        inf]
 [2.83454341 0.                inf        inf        inf]
 [1.72628826 2.7946

In [35]:
test_pairs = [(0, 4), (0, 3), (1, 4), (2, 1), (0, 1)]

for src, dst in test_pairs:
    path = reconstruct_path(final_P, src, dst)
    distance = final_D[dst, src]
    
    print(f"\n从节点 {src} 到节点 {dst}:")
    if path is not None:
        path_str = " → ".join(map(str, path))
        print(f"  路径: {path_str}")
        print(f"  距离: {distance:.6f}")
        
        # 验证路径
        if len(path) > 1:
            total = 0
            edges = []
            for i in range(len(path) - 1):
                weight = W.T[path[i+1], path[i]]
                total += weight
                edges.append(f"{path[i]}→{path[i+1]}({weight:.3f})")
            print(f"  详细: {' + '.join(edges)} = {total:.6f}")
    else:
        print(f"  状态: 不可达")
    print("-" * 60)


从节点 0 到节点 4:
  路径: 0 → 2 → 4
  距离: 2.731285
  详细: 0→2(1.726) + 2→4(1.005) = 2.731285
------------------------------------------------------------

从节点 0 到节点 3:
  路径: 0 → 2 → 3
  距离: 3.146875
  详细: 0→2(1.726) + 2→3(1.421) = 3.146875
------------------------------------------------------------

从节点 1 到节点 4:
  路径: 1 → 0 → 2 → 4
  距离: 3.799606
  详细: 1→0(1.068) + 0→2(1.726) + 2→4(1.005) = 3.799606
------------------------------------------------------------

从节点 2 到节点 1:
  状态: 不可达
------------------------------------------------------------

从节点 0 到节点 1:
  路径: 0 → 1
  距离: 2.834543
  详细: 0→1(2.835) = 2.834543
------------------------------------------------------------


## 最短路计算 带路径 torch

In [9]:
import torch
import time
from functools import wraps

# ============================================================
# 1. 性能测试装饰器
# ============================================================

def gpu_timer(func):
    """
    GPU 性能测试装饰器
    测量：GPU执行时间、迭代次数
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        # 确保之前的 GPU 操作完成
        torch.cuda.synchronize()
        
        # 创建 CUDA 事件用于精确计时
        start_event = torch.cuda.Event(enable_timing=True)
        end_event = torch.cuda.Event(enable_timing=True)
        
        # 开始计时
        start_event.record()
        
        # 执行函数
        result = func(*args, **kwargs)
        
        # 结束计时
        end_event.record()
        torch.cuda.synchronize()
        
        # 计算执行时间（毫秒）
        gpu_time = start_event.elapsed_time(end_event)
        
        # 打印性能报告
        print("\n" + "="*60)
        print(f"函数: {func.__name__}")
        print("="*60)
        print(f"GPU 执行时间: {gpu_time:.2f} ms ({gpu_time/1000:.4f} s)")
        
        # 如果返回值包含迭代次数信息
        if isinstance(result, tuple) and len(result) == 3:
            D, P, iterations = result
            print(f"迭代次数: {iterations}")
            print(f"平均每轮时间: {gpu_time/iterations:.2f} ms")
            print("="*60 + "\n")
            return D, P, iterations
        else:
            print("="*60 + "\n")
            return result
    
    return wrapper


# ============================================================
# 2. GPU 版本的 min-plus 矩阵乘法
# ============================================================

def min_plus_matmul_gpu(W, D, P_old):
    """
    GPU 加速的 (min, +) 半环矩阵乘法
    计算: D_new[i, j] = min_k { W[i, k] + D[k, j] }
    同时更新前驱矩阵 P
    
    参数:
        W: torch.Tensor, shape (N, N), 权重矩阵
        D: torch.Tensor, shape (N, N), 当前距离矩阵
        P_old: torch.Tensor, shape (N, N), 当前前驱矩阵
    
    返回:
        D_new: torch.Tensor, shape (N, N), 新的距离矩阵
        P_new: torch.Tensor, shape (N, N), 新的前驱矩阵
    """
    N = W.shape[0]
    
    # 广播计算: W[i, k] + D[k, j]
    # W.unsqueeze(1): (N, 1, N) - 对应 W[i, :, k]
    # D.unsqueeze(0): (1, N, N) - 对应 D[:, k, j]
    # 相加结果: (N, N, N) - [i, j, k] = W[i, k] + D[k, j]
    path_costs = W.unsqueeze(1) + D.unsqueeze(0)  # (N, N, N)
    
    # 沿着 k 维度(dim=2)取最小值
    # D_new[i, j] = min_k { path_costs[i, j, k] }
    # best_k[i, j] = argmin_k { path_costs[i, j, k] }
    D_new, best_k = torch.min(path_costs, dim=2)
    
    # best_k 就是前驱节点矩阵
    P_new = best_k
    
    return D_new, P_new


# ============================================================
# 3. GPU 版本的 APSP 核心算法
# ============================================================

@gpu_timer
def apsp_gpu(W, device='cuda:0', max_iterations=None, convergence_check=True):
    """
    GPU 加速的全源最短路径算法 (APSP)
    
    参数:
        W: torch.Tensor, shape (N, N)
           邻接权重矩阵，0 表示无边
        device: str, GPU 设备
        max_iterations: int or None, 最大迭代次数，None 时为 N-1
        convergence_check: bool, 是否检查收敛（可提前终止）
    
    返回:
        D: torch.Tensor, shape (N, N), 最短距离矩阵
        P: torch.Tensor, shape (N, N), 前驱节点矩阵
        iterations: int, 实际迭代次数
    """
    # 确保输入在正确的设备上，使用 float16
    W = W.to(device).to(torch.float16)
    N = W.shape[0]
    
    # 设置最大迭代次数
    if max_iterations is None:
        max_iterations = N - 1
    
    # 1. 准备权重矩阵 W_prime
    # 将 0 替换为 inf（表示无边），对角线设为 0
    W_prime = torch.where(W == 0, torch.tensor(float('inf'), dtype=torch.float16, device=device), W)
    W_prime.fill_diagonal_(0)
    
    # 2. 初始化距离矩阵 D
    D = W_prime.clone()
    
    # 3. 初始化前驱矩阵 P（使用 long 类型存储节点索引）
    P = torch.full((N, N), -1, dtype=torch.long, device=device)
    
    # 初始化前驱矩阵：有直接边的位置，前驱为源节点
    mask = W_prime < float('inf')
    P[mask] = torch.arange(N, device=device).unsqueeze(0).expand(N, N)[mask]
    
    # 4. 迭代执行 SpMM
    actual_iterations = 0
    
    for iteration in range(max_iterations):
        D_old = D.clone()
        
        # 核心操作: (D_candidates, P_candidates) = W' ⊕.⊗ (D_old, P_old)
        D_candidates, P_candidates = min_plus_matmul_gpu(W_prime, D_old, P)
        
        # 更新：只有当新路径更短时才更新
        update_mask = D_candidates < D
        D = torch.where(update_mask, D_candidates, D)
        P = torch.where(update_mask, P_candidates, P)
        
        actual_iterations += 1
        
        # 5. 收敛检查
        if convergence_check and torch.equal(D, D_old):
            break
    
    return D, P, actual_iterations


# ============================================================
# 4. CPU 上的路径重建函数
# ============================================================

def reconstruct_path(P, src, dst):
    """
    根据前驱矩阵重建从 src 到 dst 的最短路径
    在 CPU 上执行
    
    参数:
        P: torch.Tensor or numpy.ndarray, 前驱节点矩阵
        src: int, 源节点
        dst: int, 目标节点
    
    返回:
        list: 路径节点列表 [src, ..., dst]，不可达时返回 None
    """
    # 转换到 CPU 和 NumPy（如果需要）
    if torch.is_tensor(P):
        P = P.cpu().numpy()
    
    # 检查是否可达
    if P[dst, src] == -1:
        return None
    
    # 反向追溯路径
    path = []
    current = dst
    visited = set()
    max_steps = P.shape[0] + 1
    
    while current != src and len(visited) < max_steps:
        if current in visited:
            print(f"警告：检测到路径循环")
            return None
        
        visited.add(current)
        path.append(int(current))
        current = int(P[current, src])
        
        if current == -1:
            return None
    
    path.append(int(src))
    path.reverse()
    
    return path


def reconstruct_all_paths(P, src):
    """
    重建从源点 src 到所有其他节点的路径
    
    参数:
        P: torch.Tensor or numpy.ndarray, 前驱矩阵
        src: int, 源节点
    
    返回:
        dict: {dst: path_list} 映射
    """
    if torch.is_tensor(P):
        N = P.shape[0]
    else:
        N = P.shape[0]
    
    paths = {}
    for dst in range(N):
        if dst != src:
            path = reconstruct_path(P, src, dst)
            if path is not None:
                paths[dst] = path
    
    return paths


# ============================================================
# 5. 辅助函数：打印路径信息
# ============================================================

def print_path_info(W_original, D, P, src, dst):
    """
    打印从 src 到 dst 的路径详细信息
    
    参数:
        W_original: 原始权重矩阵（用于显示边权重）
        D: 距离矩阵
        P: 前驱矩阵
        src: 源节点
        dst: 目标节点
    """
    # 转到 CPU
    if torch.is_tensor(W_original):
        W_original = W_original.cpu()
    if torch.is_tensor(D):
        D = D.cpu()
    if torch.is_tensor(P):
        P = P.cpu()
    
    path = reconstruct_path(P, src, dst)
    distance = D[dst, src].item()
    
    print(f"\n从节点 {src} 到节点 {dst}:")
    
    if path is not None:
        path_str = " → ".join(map(str, path))
        print(f"  路径: {path_str}")
        print(f"  距离: {distance:.6f}")
        
        # 验证路径
        if len(path) > 1:
            total = 0
            edges = []
            for i in range(len(path) - 1):
                weight = W_original[path[i+1], path[i]].item()
                total += weight
                edges.append(f"{path[i]}→{path[i+1]}({weight:.3f})")
            print(f"  详细: {' + '.join(edges)} = {total:.6f}")
    else:
        print(f"  状态: 不可达")



In [10]:
import numpy as np

# 转换为 PyTorch Tensor
W_tensor = torch.from_numpy(W.T).float()

print("="*60)
print("开始 GPU APSP 计算")
print("="*60)

# 执行 GPU APSP（装饰器会自动打印性能信息）
D, P, iterations = apsp_gpu(
    W_tensor, 
    device='cuda:0',
    convergence_check=True
)

# 打印最终结果
print("\n--- 最终最短距离矩阵 D ---")
print(D.cpu().numpy())

print("\n--- 最终前驱节点矩阵 P ---")
print(P.cpu().numpy())



开始 GPU APSP 计算

函数: apsp_gpu
GPU 执行时间: 53.47 ms (0.0535 s)
迭代次数: 4
平均每轮时间: 13.37 ms


--- 最终最短距离矩阵 D ---
[[0.     1.068  1.727  3.148  2.73  ]
 [1.068  0.     2.795  4.492  4.08  ]
 [1.727  2.795  0.     1.265  0.799 ]
 [3.148  4.215  1.265  0.     0.6807]
 [2.73   3.8    0.799  0.6807 0.    ]]

--- 最终前驱节点矩阵 P ---
[[0 1 0 0 0]
 [1 1 1 1 1]
 [0 0 2 3 4]
 [2 2 3 3 3]
 [2 2 4 3 4]]


In [11]:
# 路径重建示例
print("\n" + "="*60)
print("路径重建示例")
print("="*60)

test_pairs = [(0, 4), (0, 3), (1, 4)]

for src, dst in test_pairs:
    print_path_info(W_tensor, D, P, src, dst)
    print("-" * 60)

# 重建从节点 0 出发的所有路径
print("\n从节点 0 到所有其他节点的路径:")
all_paths = reconstruct_all_paths(P, src=0)
for dst, path in all_paths.items():
    print(f"  到节点 {dst}: {' → '.join(map(str, path))}")


路径重建示例

从节点 0 到节点 4:
  路径: 0 → 2 → 4
  距离: 2.730469
  详细: 0→2(1.726) + 2→4(1.005) = 2.731285
------------------------------------------------------------

从节点 0 到节点 3:
  路径: 0 → 2 → 3
  距离: 3.148438
  详细: 0→2(1.726) + 2→3(1.421) = 3.146876
------------------------------------------------------------

从节点 1 到节点 4:
  路径: 1 → 0 → 2 → 4
  距离: 3.800781
  详细: 1→0(1.068) + 0→2(1.726) + 2→4(1.005) = 3.799606
------------------------------------------------------------

从节点 0 到所有其他节点的路径:
警告：检测到路径循环
  到节点 2: 0 → 2
  到节点 3: 0 → 2 → 3
  到节点 4: 0 → 2 → 4
